In [20]:
import ollama
from langchain_core.documents import Document

from utils.arxiv import create_arxiv_vectorstore
from utils.sql import DatabaseConnection

POSTGRES_USER = "username"
POSTGRES_PASSWORD = "password"
POSTGRES_HOST = "localhost"
POSTGRES_PORT = "5432"
POSTGRES_DB = "vectordb"

In [21]:
db = DatabaseConnection(
    db_name=POSTGRES_DB,
    user=POSTGRES_USER,
    password=POSTGRES_PASSWORD,
    host=POSTGRES_HOST,
    port=POSTGRES_PORT,
)

vectorstore = await create_arxiv_vectorstore(db.langchain_engine)

In [ ]:
def format_docs(docs: list[Document]) -> str:
    """
    Formata uma lista de documentos em uma string estruturada.

    Parâmetros
    ----------
    docs : list[Document]
        Lista de objetos do tipo `Document` a serem formatados.

    Retorna
    -------
    str
        Uma string contendo o conteúdo dos documentos formatados com seus IDs e separadores.
    """

    context = f"{'-'*90}\n\n"
    for doc in docs:
        context += f"DOC ID: {doc.metadata['doc_id']}\n\n"
        context += doc.page_content
        context += f"\n{'-'*90}\n\n"

    return context[:-2]


def ollama_llm(question: str, context: str) -> str:
    """
    Gera uma resposta para uma pergunta com base em um contexto fornecido.

    Parâmetros
    ----------
    question : str
        A pergunta a ser respondida.
    context : str
        O contexto no qual a resposta será baseada.

    Retorna
    -------
    str
        A resposta gerada pelo modelo de linguagem,
        em português, citando o documento de origem.
    """

    formatted_prompt = f"""
Responda a pergunta a seguir com base no contexto fornecido.
Se a resposta não estiver no contexto, diga que não sabe.
Responda em português e cite o documento de onde a informação foi retirada.

Pergunta: {question}\n\n
Contexto:\n{context}
    """
    response = ollama.chat(
        model="llama3.2:1b",
        messages=[{"role": "user", "content": formatted_prompt}],
    )
    return response["message"]["content"]


def rag(question: str) -> str:
    """
    Executa o processo de RAG para responder a uma pergunta.

    Este método utiliza um vetor de armazenamento para recuperar
    documentos relevantes com base na pergunta fornecida, formata
    os documentos recuperados e utiliza um modelo de linguagem
    para gerar uma resposta com base no contexto.

    Retorna
    -------
    str
        A resposta gerada pelo modelo de linguagem
        com base nos documentos recuperados.
    """

    retriever = vectorstore.as_retriever()
    retrieved_docs = retriever.invoke(question)
    formatted_context = format_docs(retrieved_docs)

    return ollama_llm(question, formatted_context)

In [23]:
print(rag("What is the stream computing paradigm?"))

A: O.stream computing paradigm é um modelo que permite o tratamento de dados em tempo real, distribuído e escalável, utilizando máquinas virtuais (VMs) ou servidores em nuvem. Ele tem como características principais a escalabilidade, flexibilidade e capacidade de processar grandes volumes de dados rapidamente.

Documento de origem: 
- DOC ID 750
- DOC ID 224
